# Task 1

## Question 1

In [224]:
import pandas as pd
import requests

In [225]:
## Assumption: The file is not given and needs to read from the URL

url = "https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json"
response = requests.get(url)

# Make sure the request was successful
if response.status_code == 200:
    data = response.json()
else:
    print(f"Request failed with status code {response.status_code}")

In [226]:
# Read the Excel file
country_df = pd.read_excel('Country-Code.xlsx')


In [227]:
restuarant_list = []

for page in data:
    for restaurant in page['restaurants']:
        restuarant_list.append(restaurant)

In [228]:
def get_details(restuarant_list):
    result = []
    
    for entry in restuarant_list:
        restaurant = entry.get('restaurant', {})
        
        # Get the restaurant id
        restaurant_id = restaurant.get('R', {}).get('res_id', "NA")
        
        # Get the restaurant name
        restaurant_name = restaurant.get('name', "NA")
        
        # Get the country_id
        country_id = restaurant.get('location', {}).get('country_id', "NA")
        
        # Get the city name
        city = restaurant.get('location', {}).get('city', "NA")
        
        # Get the User Rating Votes
        votes = restaurant.get('user_rating', {}).get('votes', "NA")
        
        # Get the User Aggregate Rating
        aggregate_rating = restaurant.get('user_rating', {}).get('aggregate_rating', "NA")
        
        # Get the Cuisines
        cuisines = restaurant.get('cuisines', "NA")
        
        result.append([restaurant_id, restaurant_name, country_id, city, votes, aggregate_rating, cuisines])
    
    return result
        

In [229]:
columns = ['Restaurant Id', 'Restaurant Name', 'Country', 'City', 'User Rating Votes', 'User Aggregate Rating', 'Cuisines']

restuarant_df = pd.DataFrame(get_details(restuarant_list), columns=columns)

In [230]:
restuarant_df['Country'] = restuarant_df['Country'].map(country_df.set_index('Country Code')['Country'])

In [231]:
restuarant_df.to_csv("restaurants.csv")

## Question 2

In [232]:
events_columns = ['Event Id', 'Restaurant Id', 'Restaurant Name', 'Photo URL', 'Event Title', 'Event Start Date', 'Event End Date']


In [233]:
def get_events(restuarant_list):
    result = []
    
    for entry in restuarant_list:
        restaurant = entry.get('restaurant', {})
        
        restaurant_id = restaurant.get('R', {}).get('res_id', "NA")
        
        restaurant_name = restaurant.get('name', "NA")
        
        zomato_events = restaurant.get('zomato_events', [])
        
        for item in zomato_events:
            event = item.get('event', {})
            
            event_id = event.get('event_id', "NA")
            
            photos = event.get('photos', [])
            
            photo_url = "NA"
            
            for photo in photos:
                photo_url = photo.get('photo', {}).get('url', "NA")
            
            event_title = event.get('title', "NA").strip()
            
            event_start_date = event.get('start_date', "NA")
            
            event_end_date = event.get('end_date', "NA")
            
            result.append([event_id, restaurant_id, restaurant_name, photo_url, event_title, event_start_date, event_end_date])
    
    return result
        

In [234]:
event_df = pd.DataFrame(get_events(restuarant_list), columns=events_columns)

event_df

,Event Id,Restaurant Id,Restaurant Name,Photo URL,Event Title,Event Start Date,Event End Date
0,322331,18649486,The Drunken Botanist,https://b.zmtcdn.com/data/zomato_events/photos...,BackToBasic Wednesdays !!,2019-03-06,2019-08-28
1,332812,308322,Hauz Khas Social,https://b.zmtcdn.com/data/zomato_events/photos...,Live 20/20 Match Screenings,2019-03-29,2019-05-23
2,336644,18856789,AIR- An Ivory Region,https://b.zmtcdn.com/data/zomato_events/photos...,Dhol Bhangra Night,2019-04-10,2019-04-11
3,336889,18382360,Local,NA,Cocktail Wednesday,2019-04-10,2019-04-10
4,336888,18382360,Local,NA,Cocktail Wednesday,2019-04-10,2019-04-10
...,...,...,...,...,...,...,...
179,305850,7001670,"Eight - Cordis, Auckland Hotel",https://b.zmtcdn.com/data/zomato_events/photos...,Smoky Thursdays,2019-03-01,2019-04-30
180,298162,7001670,"Eight - Cordis, Auckland Hotel",https://b.zmtcdn.com/data/zomato_events/photos...,Chocolate Tuesdays,2019-04-05,2019-08-21
181,329300,18981544,Katsu YA,https://b.zmtcdn.com/data/zomato_events/photos...,Review us on Zomato,2019-03-22,2019-04-30
182,329301,18981544,Katsu YA,https://b.zmtcdn.com/data/zomato_events/photos...,Opening Promo,2019-03-22,2019-04-30


In [235]:
event_df['Event Start Date'] = pd.to_datetime(event_df['Event Start Date'])
filtered_event_df = event_df[(event_df['Event Start Date'] >= '2019-04-01') & (event_df['Event Start Date'] <= '2019-04-30')]

filtered_event_df.to_csv('restaurant_events.csv')

## Question 3

In [236]:
def get_ratings(restuarant_list):
    results = []
    
    for entry in restuarant_list:
        restaurant = entry.get('restaurant', {})
        
        user_rating = restaurant.get('user_rating', {})
        
        rating_text = user_rating.get('rating_text', "NA")
        aggregate_rating = user_rating.get('aggregate_rating', "NA")
        
        results.append([rating_text, aggregate_rating])
        
    return results

In [237]:
ratings_df = pd.DataFrame(get_ratings(restuarant_list), columns=['rating_text', 'aggregate_rating'])
target_ratings = ['Excellent', 'Very Good', 'Good', 'Average', 'Poor']
filtered_ratings = ratings_df[ratings_df['rating_text'].isin(target_ratings)]

,rating_text,aggregate_rating
0,Very Good,4.4
1,Excellent,4.6
2,Very Good,4.1
3,Very Good,4.4
4,Very Good,4.4
...,...,...
1295,Very Good,4.2
1296,Very Good,4.0
1297,Good,3.8
1298,Excellent,4.7


In [239]:
filtered_ratings['aggregate_rating'] = pd.to_numeric(filtered_ratings['aggregate_rating'], errors='coerce')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1300 entries, 0 to 1299
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   rating_text       1300 non-null   object
 1   aggregate_rating  1300 non-null   object
dtypes: object(2)
memory usage: 20.4+ KB


In [240]:
grouped_ratings = filtered_ratings.groupby('rating_text').agg({'aggregate_rating': ['min', 'max']})
grouped_ratings = grouped_ratings.reset_index()
grouped_ratings['rating_text'] = pd.Categorical(grouped_ratings['rating_text'], categories=target_ratings, ordered=True)
sorted_ratings = grouped_ratings.sort_values('rating_text')

sorted_ratings

rating_text aggregate_rating     
                           min  max
1   Excellent              3.7  4.9
4   Very Good              4.3  4.4
2        Good              3.5  3.9
0     Average              2.5  3.4
3        Poor              0.0  2.2